In [2]:
import os
import json
from pathlib import Path
from collections import defaultdict
import re
import spacy
from spacy.training.example import Example
from spacy.tokens import DocBin
import ahocorasick
import time
import random

# Set paths
BASE_DIR = Path("..") / "data" / "sentences"
OUTPUT_DIR = Path("..") / "data" / "json"
VOCAB_DIR = Path("..") / "vocabularies"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load the input text data
preprocessed_texts = []
for file in BASE_DIR.rglob("*.txt"):
    with open(file, encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]
        preprocessed_texts.extend(lines)
print(f"📄 Loaded {len(preprocessed_texts):,} total sentences from {BASE_DIR}")


📄 Loaded 0 total sentences from ../data/sentences


In [20]:
def fix_missing_spaces(text):
    return re.sub(r'(?<=[a-zA-Z0-9])(?=[.?!])(?=[^\s])', r'\g<0> ', text)

with open(BASE_DIR / "env_data.txt", encoding="utf-8") as f:
    preprocessed_texts = [
        fix_missing_spaces(line.strip().lower())
        for line in f if line.strip()
    ]

In [4]:
def build_automaton(vocab_terms):
    A = ahocorasick.Automaton()
    for term in vocab_terms:
        A.add_word(term, term)
    A.make_automaton()
    return A

def is_inside_hyphenated_word(text, start, end):
    # Check if the match is attached to another token via a hyphen
    if start > 0 and text[start - 1] == '-':
        return True
    if end < len(text) and text[end] == '-':
        return True
    return False

def annotate_text_with_vocab(text, automaton, label):
    text_length = len(text)
    matches = []

    # Iterate using the automaton
    for end_index, term in automaton.iter(text):
        start_index = end_index - len(term) + 1

        # Whole word check
        if (start_index == 0 or not text[start_index - 1].isalnum()) and (
            end_index + 1 == text_length or not text[end_index + 1].isalnum()
        ):
            if not is_inside_hyphenated_word(text, start_index, end_index + 1):
                matches.append([start_index, end_index + 1, label])

    # Sort by start index, then longer spans first
    matches.sort(key=lambda x: (x[0], x[1] - x[0]), reverse=False)
    
    # Don't block overlaps — just collect all clean, whole-word matches
    annotations = [[start, end, label] for start, end, label in matches]

    return annotations


In [22]:
# import inflect
# from pathlib import Path

# p = inflect.engine()

# # Load vocab file
# VOCAB_DIR = Path("..") / "vocabularies"
# input_path = VOCAB_DIR / "pollutant.txt"
# output_path = VOCAB_DIR / "pollutant_with_plurals.txt"

# # Read and pluralise
# with open(input_path, "r", encoding="utf-8") as f:
#     original_terms = {line.strip().lower() for line in f if line.strip()}

# expanded_terms = set()

# for term in original_terms:
#     expanded_terms.add(term)
#     # Try to pluralise single words
#     if len(term.split()) == 1:
#         plural = p.plural(term)
#         if plural and plural != term:
#             expanded_terms.add(plural)

# # Write expanded vocab
# with open(output_path, "w", encoding="utf-8") as f:
#     for term in sorted(expanded_terms):
#         f.write(term + "\n")

# print(f"✅ Expanded {len(original_terms)} terms to {len(expanded_terms)} (with plurals added).")
# print(f"📁 Saved to: {output_path}")


✅ Expanded 196 terms to 312 (with plurals added).
📁 Saved to: ..\vocabularies\meas_with_plurals.txt


In [6]:
theme_files = [
    "measurement.txt", 
    "pollutant.txt", 
    "env_process.txt", 
    "habitat.txt", 
    "taxonomy.txt"
]


In [5]:
def resolve_overlaps(entities):
    entities = sorted(entities, key=lambda x: (x[0], -(x[1] - x[0])))
    resolved = []
    occupied = set()
    for start, end, label in entities:
        if not any(pos in occupied for pos in range(start, end)):
            resolved.append([start, end, label])
            occupied.update(range(start, end))
    return sorted(resolved, key=lambda x: x[0])


In [9]:


start_time = time.time()
text_to_annotations = defaultdict(list)

for fname in theme_files:
    theme_name = fname.replace(".txt", "")
    label = theme_name.upper()
    print(f"🔍 Annotating {label}")

    with open(VOCAB_DIR / fname, encoding="utf-8") as f:
        vocab_terms = [line.strip().lower() for line in f if line.strip()]

    automaton = build_automaton(vocab_terms)

    for i, text in enumerate(preprocessed_texts):
        annotations = annotate_text_with_vocab(text, automaton, label)
        if annotations:
            text_to_annotations[text].extend(annotations)
            text_to_annotations[text] = resolve_overlaps(text_to_annotations[text])
        if (i + 1) % 100000 == 0:
            print(f"Processed {i + 1:,}/{len(preprocessed_texts):,} texts...")

# Save final output
annotated_path = OUTPUT_DIR / "training_data.jsonl"
with open(annotated_path, "w", encoding="utf-8") as f:
    for text, annotations in text_to_annotations.items():
        json.dump({"text": text, "label": annotations}, f, ensure_ascii=False)
        f.write("\n")

end_time = time.time()
print(f"✅ Done! Annotated {len(text_to_annotations):,} unique texts.")
print(f"📁 Saved to: {annotated_path}")
print(f"⏱ Total time: {end_time - start_time:.2f} seconds")


🔍 Annotating MEASUREMENT
Processed 100,000/2,910,834 texts...
Processed 200,000/2,910,834 texts...
Processed 300,000/2,910,834 texts...
Processed 400,000/2,910,834 texts...
Processed 500,000/2,910,834 texts...
Processed 600,000/2,910,834 texts...
Processed 700,000/2,910,834 texts...
Processed 800,000/2,910,834 texts...
Processed 900,000/2,910,834 texts...
Processed 1,000,000/2,910,834 texts...
Processed 1,100,000/2,910,834 texts...
Processed 1,200,000/2,910,834 texts...
Processed 1,300,000/2,910,834 texts...
Processed 1,400,000/2,910,834 texts...
Processed 1,500,000/2,910,834 texts...
Processed 1,600,000/2,910,834 texts...
Processed 1,700,000/2,910,834 texts...
Processed 1,800,000/2,910,834 texts...
Processed 1,900,000/2,910,834 texts...
Processed 2,000,000/2,910,834 texts...
Processed 2,100,000/2,910,834 texts...
Processed 2,200,000/2,910,834 texts...
Processed 2,300,000/2,910,834 texts...
Processed 2,400,000/2,910,834 texts...
Processed 2,500,000/2,910,834 texts...
Processed 2,600,00

In [10]:
import random
import json
from pathlib import Path

INPUT_FILE = Path("..") / "data" / "json" / "training_data.jsonl"
OUTPUT_FILE = Path("..") / "data" / "json" / "sample_for_manual_testing.jsonl"

all_data = []

# Load safely, skip blank or bad lines
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            all_data.append(data)
        except json.JSONDecodeError:
            print("⚠️ Skipped malformed line")

# Sample
sampled = random.sample(all_data, min(1000, len(all_data)))

# Save sample
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for item in sampled:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Sampled {len(sampled)} items to {OUTPUT_FILE}")


✅ Sampled 1000 items to ../data/json/sample_for_manual_testing.jsonl


In [ ]:


# --- Directory Setup ---
BASE_DIR = Path("data") / "raw_data"
OUTPUT_DIR = Path("data") / "processed_data"
annotated_path = OUTPUT_DIR / "training_data.jsonl"
cleaned_path = OUTPUT_DIR / "cleaned_training_data.jsonl"

# --- Helper Functions ---
def has_overlapping_entities(entities):
    sorted_entities = sorted(entities, key=lambda x: x[0])
    for i in range(len(sorted_entities) - 1):
        current_start, current_end, _ = sorted_entities[i]
        next_start, _, _ = sorted_entities[i + 1]
        if current_end > next_start:
            return True
    return False

def resolve_overlaps(entities):
    entities = sorted(entities, key=lambda x: (x[0], -(x[1] - x[0])))
    resolved = []
    occupied = set()
    for start, end, label in entities:
        if not any(pos in occupied for pos in range(start, end)):
            resolved.append([start, end, label])
            occupied.update(range(start, end))
    return sorted(resolved, key=lambda x: x[0])

# --- SpaCy Setup ---
nlp = spacy.blank("en")
nlp.max_length = 5_000_000

# --- Load Annotated Data ---
with open(annotated_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line.strip()) for line in f]

valid_data = []
invalid_data = []

for i, example in enumerate(raw_data):
    text = example["text"]
    annotations = example["label"]

    if has_overlapping_entities(annotations):
        annotations = resolve_overlaps(annotations)

    doc = nlp(text)
    try:
        Example.from_dict(doc, {"entities": annotations})
        valid_data.append({"text": text, "label": annotations})
    except Exception as e:
        invalid_data.append({
            "index": i,
            "error": str(e),
            "text": text,
            "label": annotations
        })

# --- Save Cleaned Data ---
with open(cleaned_path, "w", encoding="utf-8") as f:
    for item in valid_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"Cleaned {len(valid_data)} valid samples.")
print(f"Skipped {len(invalid_data)} invalid samples.")
print(f"Saved cleaned annotations to: {cleaned_path}")


In [13]:
import spacy
from pathlib import Path

# Paths
BASE_DIR = Path("..") / "data" / "raw_data"
OUTPUT_DIR = Path("..") / "data" / "processed_data"
VOCAB_DIR = Path("..") / "vocabularies"

input_path = VOCAB_DIR / "taxonomy.txt"
output_path = VOCAB_DIR / "taxonomy_lemmatized.txt"

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Read original taxonomy terms
with open(input_path, "r", encoding="utf-8") as f:
    terms = [line.strip().lower() for line in f if line.strip()]

lemmatised_terms = set()

for term in terms:
    doc = nlp(term)
    lemma = " ".join([token.lemma_ for token in doc])
    lemmatised_terms.add(lemma)

# Sort and save
lemmatised_sorted = sorted(lemmatised_terms)

with open(output_path, "w", encoding="utf-8") as f:
    for term in lemmatised_sorted:
        f.write(term + "\n")

print(f"Lemmatized {len(terms)} terms down to {len(lemmatised_terms)} unique ones.")
print(f"Saved to: {output_path}")


Lemmatized 4961 terms down to 4554 unique ones.
Saved to: ..\vocabularies\taxonomy_lemmatized.txt


# Training an example model

In [11]:
OUTPUT_DIR = Path("..") / "data" / "json"
VOCAB_DIR = Path("..") / "vocabularies"
SPACY_DIR = Path("..") / "data" / "spacy_data"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SPACY_DIR, exist_ok=True)

INPUT_FILE = OUTPUT_DIR / "training_data.jsonl"
TRAIN_JSONL = SPACY_DIR / "train.jsonl"
DEV_JSONL = SPACY_DIR / "dev.jsonl"

In [12]:

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    all_data = [json.loads(line) for line in f if line.strip()]

random.shuffle(all_data)

split = int(len(all_data) * 0.9)
train_data, dev_data = all_data[:split], all_data[split:]

with open(TRAIN_JSONL, "w", encoding="utf-8") as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

with open(DEV_JSONL, "w", encoding="utf-8") as f:
    for item in dev_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"Saved {len(train_data)} train and {len(dev_data)} dev examples.")


Saved 560382 train and 62265 dev examples.


In [13]:


def convert_to_docbin(input_path, output_path, nlp):
    doc_bin = DocBin()
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            doc = nlp.make_doc(item["text"])
            ents = []
            for start, end, label in item["label"]:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)
    doc_bin.to_disk(output_path)

# Load blank model for tokenization
nlp = spacy.blank("en")

convert_to_docbin(Path("../data/spacy_data/train.jsonl"), Path("../data/spacy_data/train.spacy"), nlp)
convert_to_docbin(Path("../data/spacy_data/dev.jsonl"), Path("../data/spacy_data/dev.spacy"), nlp)

print("✅ Converted to .spacy format")


✅ Converted to .spacy format


In [14]:
!python -m spacy init config ../config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
../config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output models/env_ner


In [14]:
# Load your trained model (adjust path as needed)
nlp = spacy.load("../models/env_ner/model-best")

# Long test text with mixed taxonomy terms (common + scientific)
text = """
Researchers in the Galápagos Islands have observed an unusual breeding pattern in Chelonoidis hoodensis, the Española giant tortoise, within fragmented dry forest patches. This habitat is increasingly threatened by rising nitrogen oxide levels, a pollutant linked to agricultural runoff.

Meanwhile, in the peat bogs of northern Scotland, the rare moss Sphagnum balticum is showing signs of resilience under fluctuating temperature gradients — a direct consequence of erratic jet stream shifts, an environmental process becoming more frequent due to Arctic warming.

An alarming spike in mercury concentration was recorded in wetland fish near the Mekong delta, where biodiversity is under pressure. The riverine habitat has also experienced abnormal algal blooms — a sign of nutrient loading, which is a key measurement in eutrophication studies.

In southern India, the tropical deciduous forests host a variety of mammals including the rusty-spotted cat (*Prionailurus rubiginosus*), one of the smallest wild felines, which now faces threats due to increasing particulate matter in the air. This pollutant, PM2.5, has exceeded safe thresholds in over 40% of sampled areas.

Glaciologists reported that meltwater runoff from the Vatnajökull glacier has doubled in the past decade — a clear signal of accelerated cryospheric change, an environmental process linked to anthropogenic CO₂ emissions. Concurrently, the lichen *Xanthoria parietina* has colonised newly exposed rock surfaces, suggesting rapid ecological succession in formerly ice-covered zones.

In a monitoring project across coral reef habitats in the Seychelles, elevated salinity and pH fluctuations were documented — measurements that correlate strongly with declining populations of *Symbiodinium*, a genus of symbiotic algae critical for coral health. Oil residues, another pollutant from passing tankers, were also found on reef sediments.

A surprising comeback of the great bustard (*Otis tarda*) in steppe habitats of eastern Spain was attributed to habitat restoration programmes and reduced pesticide use. The species had previously vanished due to soil degradation — a gradual environmental process driven by overgrazing and monoculture practices.
"""

# Process the text with your model
doc = nlp(text.lower())

# Print predicted entities and their labels
for ent in doc.ents:
    print(f"{ent.text} -> {ent.label_}")


islands -> HABITAT
tortoise -> TAXONOMY
forest -> HABITAT
habitat -> HABITAT
bogs -> HABITAT
temperature -> MEASUREMENT
mercury -> POLLUTANT
wetland -> HABITAT
fish -> TAXONOMY
delta -> HABITAT
habitat -> HABITAT
forests -> HABITAT
mammals -> TAXONOMY
cat -> TAXONOMY
particulate matter -> POLLUTANT
glacier -> HABITAT
rock -> HABITAT
coral reef -> HABITAT
habitats -> HABITAT
salinity -> MEASUREMENT
reef -> HABITAT
habitats -> HABITAT
habitat -> HABITAT
pesticide -> POLLUTANT
overgrazing -> HABITAT


In [5]:
# Load your trained model (adjust path as needed)
nlp = spacy.load("../models/env_ner/model-best")

# Long test text with mixed taxonomy terms (common + scientific)
text = """
The Swiss village of Blatten has been partially destroyed after a huge chunk of glacier crashed down into the valley.

Although the village had been evacuated some days ago because of fears the Birch glacier was disintegrating, one person has been reported missing, and many homes have been completely flattened.

Blatten's mayor, Matthias Bellwald, said "the unimaginable has happened" but promised the village still had a future.

Local authorities have requested support from the Swiss army's disaster relief unit and members of the Swiss government are on their way to the scene.

The disaster that has befallen Blatten is the worst nightmare for communities across the Alps.

The village's 300 inhabitants had to leave their homes on 19 May after geologists monitoring the area warned that the glacier appeared unstable. Now many of them may never be able to return.

Appearing to fight back tears, Bellwald said: "We have lost our village, but not our heart. We will support each other and console each other. After a long night, it will be morning again."

The Swiss government has already promised funding to make sure residents can stay, if not in the village itself, at least in the locality.

However, Raphaël Mayoraz, head of the regional office for Natural Hazards, warned that further evacuations in the areas close to Blatten might be necessary.

Climate change is causing the glaciers - frozen rivers of ice - to melt faster and faster, and the permafrost, often described as the glue that holds the high mountains together, is also thawing.

Drone footage showed a large section of the Birch glacier collapsing at about 15:30 (14:30 BST) on Wednesday. The avalanche of mud that swept over Blatten sounded like a deafening roar, as it swept down into the valley leaving an enormous cloud of dust.

Glaciologists monitoring the thaw have warned for years that some alpine towns and villages could be at risk, and Blatten is not even the first to be evacuated.

In eastern Switzerland, residents of the village of Brienz were evacuated two years ago because the mountainside above them was crumbling.

Since then, they have only been permitted to return for short periods.

In 2017, eight hikers were killed, and many homes destroyed, when the biggest landslide in over a century came down close to the village of Bondo.

The most recent report into the condition of Switzerland's glaciers suggested they could all be gone within a century, if global temperatures could not be kept within a rise of 1.5C above pre-industrial levels, agreed ten years ago by almost 200 countries under the Paris climate accord.

Many climate scientists suggest that target has already been missed, meaning the glacier thaw will continue to accelerate, increasing the risk of flooding and landslides, and threatening more communities like Blatten.
"""


# Process the text with your model
doc = nlp(text.lower())

# Print predicted entities and their labels
for ent in doc.ents:
    print(f"{ent.text} -> {ent.label_}")


chunk -> TAXONOMY
glacier -> HABITAT
valley -> HABITAT
glacier -> HABITAT
glacier -> HABITAT
climate change -> ENV_PROCESS
rivers -> HABITAT
glue -> POLLUTANT
mountains -> HABITAT
glacier -> HABITAT
avalanche -> ENV_PROCESS
valley -> HABITAT
cloud -> ENV_PROCESS
dust -> POLLUTANT
landslide -> HABITAT
glaciers -> HABITAT
temperatures -> MEASUREMENT
paris -> TAXONOMY
climate -> ENV_PROCESS
climate -> ENV_PROCESS
glacier -> HABITAT
landslides -> HABITAT
